# Comparison using kerchunk / datatree

In [ ]:
from datatree import DataTree
import dask.bag as db
import xarray as xr
import pandas as pd
import coiled
from utils import load_elev, generate_WBGT

In [ ]:
cluster = coiled.Cluster(n_workers=10)
client = cluster.get_client()

In [ ]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_nested.csv"
)
# Select only ssp245 && HISTORICAL!
ssp245_historical_catalog = cat_df[cat_df["ID"].str.contains("ssp245|historical")]

# Subset 20 GCMs for time comparison
ssp245_historical_catalog = ssp245_historical_catalog.iloc[0:20]

# Convert the DataFrame into a dictionary
catalog = ssp245_historical_catalog.set_index("ID").T.to_dict("records")[0]

In [ ]:
def load_ref_ds(gcm_scenario: str, url: str):
    storage_options = {
        "remote_protocol": "s3",
        "target_protocol": "s3",
        "lazy": True,
        "skip_instance_cache": True,
    }  # options passed to fsspec
    open_dataset_options = {"chunks": {}}  # opens passed to xarray
    ds = xr.open_dataset(
        url,
        engine="kerchunk",
        storage_options=storage_options,
        open_dataset_options=open_dataset_options,
    )

    if set(
        [
            "huss",
            "tasmax",
            "tas",
        ]
    ).issubset(set(list(ds))):
        ds = ds[["huss", "tasmax", "tas"]]
        # adding the gcm/scenario combo to attrs for later down the pipeline
        ds.attrs["gcm_scenario"] = gcm_scenario
        return {gcm_scenario: ds}


# convert catalog dict to tuples for dask bag starmap
catalog_tuple_list = list(catalog.items())
# Create a bag from tuple catalog
cat_bag = db.from_sequence(catalog_tuple_list)
# apply load_ref_ds to catalog tuple bag
task_bag = filter(None, cat_bag.starmap(load_ref_ds).compute())

# convert list of dicts to dict
catalog_computed = {k: v for element in task_bag for k, v in element.items()}

In [ ]:
# Create datatree object from computed dictionary
dt = DataTree.from_dict(catalog_computed)

# load elev dataset for WBGT calc
elev = load_elev()

In [ ]:
def calc_wbgt(ds):
    ds = ds.isel(time=slice(0, 365))
    output = (
        f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
        f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
    )
    return generate_WBGT(ds=ds, output_fpath=output)


ds_list = [ds.to_dataset() for ds in dt.leaves if ds.dims]
bag = db.from_sequence(ds_list, npartitions=len(ds_list)).map(calc_wbgt)

In [ ]:
# calls compute on dask bag to trigger computation
bag.compute()

In [ ]:
client.shutdown()